In [117]:
import sys
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import pickle
import re
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import datasets
from datasets import Dataset

import warnings # выключение отображение warning-ов
import transformers

warnings.filterwarnings('ignore')
print(transformers.__version__)

In [93]:
# !{sys.executable} -m pip install datasets

In [3]:
dataset_path = 'topics'
topic_names = ['text']
for x in os.listdir(dataset_path):
    topic_names.append(x)
# topic_names

In [4]:
dataset = pd.DataFrame(columns=topic_names)
dataset

,text,alone,america,angel,anger,animal,baby,beach,beautiful,beauty,...,trust,truth,war,warning,water,weather,wedding,winter,work,world


In [8]:
for topic in tqdm(os.listdir(dataset_path)):
    for file in os.listdir(f'{dataset_path}/{topic}'):
        with open(f'{dataset_path}/{topic}/{file}', encoding = 'utf-8') as f:
            str = f.read()
            index = dataset.index[dataset['text'] == str]
            # print(index.size)
            if index.size != 0:  #  если текст есть в датасете
                # print('not None')
                dataset[topic][index] = 1
            else:   # если текст отсутствует в датасете
                
                dataset = dataset.append({'text': str, topic: 1}, ignore_index=True)
                # print(file)

100%|██████████| 144/144 [2:31:58<00:00, 63.32s/it] 


In [9]:
dataset

,text,alone,america,angel,anger,animal,baby,beach,beautiful,beauty,...,trust,truth,war,warning,water,weather,wedding,winter,work,world
0,loneliness\nhis childhood\nwarms him up\nlonel...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sit in a chair\nalone\nPeople I thought were\n...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Tell me why it has to be this way\nwhy I must ...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"A debtor to mercy alone, of covenant mercy I s...",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A year has been passed since I left home\nAgai...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13732,"Wake not for the world-heard thunder,\nNor the...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
13733,"We may roam through this world, like a child a...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
13734,When you were born in this world\nEveryone lau...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
13735,"WORLD, take good notice, silver stars fading,\...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [11]:
dataset_fillna = dataset.fillna(0)

In [24]:
dataset

,text,passion,daughter,weather,animal,sea,innocence,change,angel,brother,...,freedom,dark,hate,music,warning,teacher,peace,chicago,butterfly,murder
0,"Your passion is permanently etched on my soul,...",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"When man had ceased to utter his lament, A god...",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"O Gentle Love, ungentle for thy deed, Thou mak...",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,PASSION TOUCHED HER LIPS Passion touched her l...,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,You eclipse me & I have stained the Sun with b...,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13731,Violence is nobody s leaven. Murder will not g...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
13732,St. Patrick s Church was sparsely filled For F...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
13733,No one sleeps more beautifully than you. But I...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
13734,"Pastor in the pulpit, the Sanctuary, supposedl...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [13]:
with open(r'dataset_fillna.pkl', 'wb') as f:
    pickle.dump(dataset_fillna, f)

Загрузка датасета

In [32]:
with open(r'dataset_fillna.pkl', 'rb') as f:
    dataset = pickle.load(f)

Обработка датасета

In [33]:
EXCLUDE_SYMBOLS_STR = u''.join(['№', '«', 'ђ', '°', '±', 'ћ', '‰', '»', 'ѓ', 'µ', '·', 'ґ', 'њ', 'ї', 'џ', 'є', '‹',
                                '‡', '†', '¶', 'ќ', '€', '“', 'ў', '§', '„', '”', '\ufeff', 'љ', '›', '•', '—', 
                                '\x7f', '\xad', '¤', '\xa0', '\u200e', 'Š', 'ō', 'ä', '™', '×', '\'', '~'])

regex_symb = re.compile('[%s]' % re.escape(EXCLUDE_SYMBOLS_STR))

for row in range(dataset.shape[0]):
    dataset["text"][row] = regex_symb.sub(' ',  dataset["text"][row]) # убираем дополнительные символы
    dataset["text"][row] = dataset["text"][row].replace("\n", " ") # убираем \n
    dataset["text"][row] = re.sub(' +', ' ', dataset["text"][row]) # оставляем максимум 1 пробел
    dataset["text"][row] = dataset["text"][row].strip() # удаляем лишние пробелы в начале и конце статей
    

In [192]:
with open(r'dataset_processed.pkl', 'wb') as f:
    pickle.dump(dataset, f)

In [34]:
dataset["text"][1]

'When man had ceased to utter his lament, A god then let me tell my tale of sorrow. WHAT hope of once more meeting is there now In the still-closed blossoms of this day? Both heaven and hell thrown open seest thou; What wav ring thoughts within the bosom play No longer doubt! Descending from the sky, She lifts thee in her arms to realms on high. And thus thou into Paradise wert brought, As worthy of a pure and endless life; Nothing was left, no wish, no hope, no thought, Here was the boundary of thine inmost strife: And seeing one so fair, so glorified, The fount of yearning tears was straightway dried. No motion stirr d the day s revolving wheel, In their own front the minutes seem d to go; The evening kiss, a true and binding seal, Ne er changing till the morrow s sunlight glow. The hours resembled sisters as they went. Yet each one from another different. The last hour s kiss, so sadly sweet, effac d A beauteous network of entwining love. Now on the threshold pause the feet, now has

In [35]:
dataset.drop("text", axis=1)

,passion,daughter,weather,animal,sea,innocence,change,angel,brother,world,...,freedom,dark,hate,music,warning,teacher,peace,chicago,butterfly,murder
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13731,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
13732,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
13733,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
13734,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


Разделение на датасет для тренировки и проверки

In [86]:
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

In [50]:
batch_size = 16

In [185]:
device = torch.device("cpu")

Topics

In [56]:
labels = dataset.columns[1:].values
labels

array(['passion', 'daughter', 'weather', 'animal', 'sea', 'innocence',
       'change', 'angel', 'brother', 'world', 'paris', 'work', 'soldier',
       'together', 'christmas', 'dance', 'sleep', 'happy', 'faith',
       'school', 'courage', 'loss', 'romance', 'nature', 'identity',
       'car', 'greed', 'lost', 'june', 'truth', 'destiny', 'trust',
       'children', 'believe', 'culture', 'silver', 'frog', 'red', 'food',
       'sun', 'romantic', 'time', 'lust', 'mother', 'heaven', 'memory',
       'crazy', 'money', 'racism', 'girl', 'home', 'evil', 'future',
       'running', 'carpe diem', 'sorrow', 'january', 'funeral', 'travel',
       'spring', 'fire', 'poem', 'suicide', 'remember', 'poetry',
       'football', 'green', 'laughter', 'cinderella', 'poverty', 'today',
       'birth', 'happiness', 'life', 'son', 'winter', 'hair', 'rose',
       'rainbow', 'god', 'star', 'house', 'power', 'sympathy',
       'depression', 'snake', 'night', 'sick', 'despair', 'husband',
       'rain', 'lov

In [60]:
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

BERT tokenizer

In [66]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased") # create a tokenizer that corresponds to BERT.

Data preprocess

In [166]:
def preprocess_data(examples):
    # take a batch of texts
    text = examples["text"]
    # encode them
    encoding = tokenizer(text, padding="max_length", truncation=True, max_length=256)
    # add labels
    labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
    # create numpy array of shape (batch_size, num_labels)
    labels_matrix = np.zeros((len(text), len(labels)))
    # fill numpy array
    for idx, label in enumerate(labels):
        labels_matrix[:, idx] = labels_batch[label]

    encoding["labels"] = labels_matrix.tolist()

    return encoding

In [97]:
dd = datasets.DatasetDict({"train":Dataset.from_pandas(train_dataset),"test":Dataset.from_pandas(test_dataset)})
dd

DatasetDict({
    train: Dataset({
        features: ['text', 'passion', 'daughter', 'weather', 'animal', 'sea', 'innocence', 'change', 'angel', 'brother', 'world', 'paris', 'work', 'soldier', 'together', 'christmas', 'dance', 'sleep', 'happy', 'faith', 'school', 'courage', 'loss', 'romance', 'nature', 'identity', 'car', 'greed', 'lost', 'june', 'truth', 'destiny', 'trust', 'children', 'believe', 'culture', 'silver', 'frog', 'red', 'food', 'sun', 'romantic', 'time', 'lust', 'mother', 'heaven', 'memory', 'crazy', 'money', 'racism', 'girl', 'home', 'evil', 'future', 'running', 'carpe diem', 'sorrow', 'january', 'funeral', 'travel', 'spring', 'fire', 'poem', 'suicide', 'remember', 'poetry', 'football', 'green', 'laughter', 'cinderella', 'poverty', 'today', 'birth', 'happiness', 'life', 'son', 'winter', 'hair', 'rose', 'rainbow', 'god', 'star', 'house', 'power', 'sympathy', 'depression', 'snake', 'night', 'sick', 'despair', 'husband', 'rain', 'love', 'river', 'city', 'childhood', 'sister',

In [167]:
encoded_dataset = dd.map(preprocess_data, batched=True, remove_columns=dd['train'].column_names)

Пример

In [168]:
example = encoded_dataset['train'][1]
# example = dd['train'][1]
print(example.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [169]:
example

{'input_ids': [101,
  1045,
  2156,
  1996,
  3548,
  10998,
  2006,
  4823,
  1037,
  3147,
  2162,
  2299,
  2006,
  1996,
  2346,
  2027,
  2907,
  3031,
  2037,
  4409,
  2004,
  1996,
  3554,
  3632,
  2006,
  2035,
  2154,
  2146,
  2954,
  1010,
  2233,
  1010,
  1998,
  2954,
  5607,
  1010,
  6614,
  1998,
  5607,
  2157,
  3251,
  1996,
  3103,
  12342,
  2015,
  1010,
  2023,
  2003,
  2162,
  2023,
  2003,
  2162,
  3251,
  1996,
  15811,
  10364,
  2015,
  2023,
  2003,
  2162,
  2073,
  2057,
  3102,
  2256,
  2814,
  1010,
  5208,
  1998,
  3428,
  2005,
  2027,
  2024,
  2256,
  6716,
  2005,
  2613,
  2065,
  2027,
  5454,
  2114,
  2149,
  2005,
  2178,
  2027,
  2175,
  2000,
  2162,
  2138,
  1997,
  1037,
  9210,
  1997,
  2576,
  2273,
  2040,
  3233,
  4142,
  1025,
  1999,
  14908,
  4822,
  2145,
  3173,
  2037,
  25636,
  1996,
  3548,
  2954,
  2127,
  2053,
  2028,
  4832,
  2039,
  4902,
  999,
  2954,
  1010,
  2233,
  1010,
  1998,
  2954,
  5607,
  1010,

In [170]:
tokenizer.decode(example['input_ids'])

'[CLS] i see the soldiers marching on singing a cold war song on the road they hold onto their guns as the fighting goes on all day long fight, march, and fight shoot, aim and shoot right whether the sun shines, this is war this is war whether the rains pours this is war where we kill our friends, sisters and brothers for they are our enemies for real if they choose against us for another they go to war because of a handful of political men who stand alive ; in erect offices still holding their pens the soldiers fight until no one stands up anymore! fight, march, and fight shoot, aim and shoot right whether the sun shines, this is war this is war whether the rains pours www. sylviachidi. com [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

In [158]:
example['labels']

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [171]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['war']

Изменение формата

In [172]:
encoded_dataset.set_format("torch")

# Model

In [186]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

## Train the model!

We are going to train the model using HuggingFace's Trainer API. This requires us to define 2 things: 

* `TrainingArguments`, which specify training hyperparameters. All options can be found in the [docs](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments). Below, we for example specify that we want to evaluate after every epoch of training, we would like to save the model every epoch, we set the learning rate, the batch size to use for training/evaluation, how many epochs to train for, and so on.
* a `Trainer` object (docs can be found [here](https://huggingface.co/transformers/main_classes/trainer.html#id1)).

В качестве метрики для оценивания будем использовать F1 score, так как ...

In [119]:
batch_size = 8 # размер батча

In [187]:
args = TrainingArguments(
    f"bert-finetuned-sem_eval-english", # папка для сохранения контрольных точек
    evaluation_strategy = "epoch", # оценивание во время обучения
    save_strategy = "epoch", # сохранение модели (после эпохи)
    learning_rate=2e-5, # lr для AdamW оптимизатора
    per_device_train_batch_size=batch_size, # размер батча при тренирвке
    per_device_eval_batch_size=batch_size, # размер батча при оценивании
    num_train_epochs=5, # число эпох обучения
    weight_decay=0.01, # вес для слоёв
    load_best_model_at_end=True, # загружать ли лучшую модель
    metric_for_best_model="f1", # метрика, на основе которой выберется лучшая модель
    dataloader_drop_last=True
)

We are also going to compute metrics while training. For this, we need to define a compute_metrics function, that returns a dictionary with the desired metric values.

In [164]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, precision_score, recall_score
from transformers import EvalPrediction
import torch
    
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions)) # переводим вероятности в область от 0 до 1

    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1 # если вероятность > treshold, то ставим 1 в предсказании (остальное 0)

    y_true = labels # реальное значение топиков
    
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)

    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy,
               'precision': precision,
               'recall': recall
              }
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    print("p.predictions:", p.predictions)
    print("preds:", preds)
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [179]:
# encoded_dataset['train'][0]['labels'].type()

'torch.FloatTensor'

In [188]:
encoded_dataset['train']['input_ids'][1]

tensor([  101,  1045,  2156,  1996,  3548, 10998,  2006,  4823,  1037,  3147,
         2162,  2299,  2006,  1996,  2346,  2027,  2907,  3031,  2037,  4409,
         2004,  1996,  3554,  3632,  2006,  2035,  2154,  2146,  2954,  1010,
         2233,  1010,  1998,  2954,  5607,  1010,  6614,  1998,  5607,  2157,
         3251,  1996,  3103, 12342,  2015,  1010,  2023,  2003,  2162,  2023,
         2003,  2162,  3251,  1996, 15811, 10364,  2015,  2023,  2003,  2162,
         2073,  2057,  3102,  2256,  2814,  1010,  5208,  1998,  3428,  2005,
         2027,  2024,  2256,  6716,  2005,  2613,  2065,  2027,  5454,  2114,
         2149,  2005,  2178,  2027,  2175,  2000,  2162,  2138,  1997,  1037,
         9210,  1997,  2576,  2273,  2040,  3233,  4142,  1025,  1999, 14908,
         4822,  2145,  3173,  2037, 25636,  1996,  3548,  2954,  2127,  2053,
         2028,  4832,  2039,  4902,   999,  2954,  1010,  2233,  1010,  1998,
         2954,  5607,  1010,  6614,  1998,  5607,  2157,  3251, 

In [189]:
#forward pass
outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))
outputs

SequenceClassifierOutput(loss=tensor(0.6976, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[ 0.1693, -0.0342,  0.2474, -0.0891,  0.1187,  0.0364,  0.2213, -0.1987,
         -0.4812, -0.2975,  0.1478, -0.3266, -0.0094,  0.1391, -0.0206, -0.0684,
          0.5259, -0.1974,  0.0869, -0.3148,  0.3708,  0.0456,  0.2298,  0.1605,
          0.1040, -0.6016, -0.1807, -0.2613, -0.3627,  0.6999,  0.2443, -0.2586,
          0.1490, -0.1866,  0.2348, -0.6191, -0.1054,  0.3113,  0.1990,  0.1593,
         -0.1383, -0.2160, -0.0870, -0.3349,  0.3600, -0.2608,  0.2605, -0.5622,
          0.2469, -0.2694,  0.0480,  0.2376, -0.0070, -0.3804,  0.0539,  0.1944,
          0.0988,  0.0907, -0.0454, -0.3037,  0.3083, -0.0190, -0.0896, -0.1437,
          0.3018, -0.3925,  0.1806, -0.1201,  0.1375, -0.3407,  0.3874,  0.1513,
         -0.0252,  0.0338, -0.4169,  0.6547,  0.0755,  0.2609, -0.5953, -0.4057,
          0.1371, -0.3547, -0.2773,  0.0252, -0.3120, -0.0121, -0.1351,  0.0286,
       

# Training

In [190]:
trainer = Trainer(
    model, # модель для обучения
    args, # аргументы для обучения
    train_dataset=encoded_dataset["train"], # датасет, используемый при обучении
    eval_dataset=encoded_dataset["test"], # датасет, используемый для оценивания
#     tokenizer=tokenizer, # токенизатор
    compute_metrics=compute_metrics # функция для подсчёта метрик при оценивании
)

In [191]:
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

# Оценивание (вывод метрик)

In [127]:
trainer.evaluate()

{'eval_loss': 0.04290616884827614,
 'eval_f1': 0.0,
 'eval_roc_auc': 0.5,
 'eval_accuracy': 0.0,
 'eval_runtime': 60.6555,
 'eval_samples_per_second': 45.305,
 'eval_steps_per_second': 5.671,
 'epoch': 5.0}

# Проверка работоспособности

In [128]:
text = "I'm happy I can finally train a model for multi-label classification"

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

The logits that come out of the model are of shape (batch_size, num_labels). As we are only forwarding a single sentence through the model, the batch_size equals 1. The logits is a tensor that contains the (unnormalized) scores for every individual label.

In [129]:
logits = outputs.logits
logits.shape

torch.Size([1, 144])

To turn them into actual predicted labels, we first apply a sigmoid function independently to every score, such that every score is turned into a number between 0 and 1, that can be interpreted as a "probability" for how certain the model is that a given class belongs to the input text.

Next, we use a threshold (typically, 0.5) to turn every probability into either a 1 (which means, we predict the label for the given example) or a 0 (which means, we don't predict the label for the given example).

In [131]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
print(probs)
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

tensor([0.0072, 0.0081, 0.0105, 0.0096, 0.0081, 0.0085, 0.0091, 0.0079, 0.0085,
        0.0090, 0.0093, 0.0089, 0.0073, 0.0088, 0.0096, 0.0092, 0.0084, 0.0104,
        0.0075, 0.0085, 0.0075, 0.0081, 0.0092, 0.0089, 0.0085, 0.0082, 0.0088,
        0.0083, 0.0093, 0.0097, 0.0082, 0.0086, 0.0096, 0.0082, 0.0085, 0.0077,
        0.0084, 0.0082, 0.0083, 0.0105, 0.0072, 0.0080, 0.0092, 0.0075, 0.0096,
        0.0102, 0.0089, 0.0092, 0.0074, 0.0081, 0.0089, 0.0088, 0.0075, 0.0089,
        0.0091, 0.0093, 0.0086, 0.0085, 0.0082, 0.0072, 0.0085, 0.0083, 0.0093,
        0.0081, 0.0069, 0.0087, 0.0082, 0.0074, 0.0076, 0.0101, 0.0083, 0.0080,
        0.0079, 0.0097, 0.0093, 0.0091, 0.0085, 0.0098, 0.0084, 0.0078, 0.0090,
        0.0078, 0.0078, 0.0091, 0.0099, 0.0066, 0.0086, 0.0091, 0.0087, 0.0088,
        0.0084, 0.0073, 0.0085, 0.0090, 0.0087, 0.0087, 0.0083, 0.0086, 0.0097,
        0.0061, 0.0079, 0.0087, 0.0076, 0.0083, 0.0081, 0.0083, 0.0089, 0.0098,
        0.0072, 0.0071, 0.0098, 0.0084, 